# Richards equation

Let $\Omega=(0,2)\times(0,3)$ with boundary $\partial \Omega$ and outward unit normal ${\nu}$. Given 
$k$ the matrix permeability, we want to solve the following problem: find $({q}, h)$ such that
$$
\partial_t \theta (\psi) - \nabla \cdot (K \nabla h) = 0 \text{ in } \Omega \times (0,T)
$$
with boundary conditions:

$$
h(t,x,z)=
\left\{
\begin{array}{ll}
\begin{array}{l} 
1+2.2 \frac{t}{\Delta t_D}, \text{ on } \Gamma_{D_1}, t\leq\Delta t_D\\
3.2, \text{ on } \Gamma_{D_1}, t>\Delta t_D\\
1, \text{ on } \Gamma_{D_2}
\end{array}
\end{array}
\right., \qquad \nu \cdot K \nabla h = 0 \text{ on } \Gamma_N \qquad h(0,x,z) = 1 \text{ on } \Omega$$
and
$$
\Gamma_{D_1} = \left\{ (x,z) \in \partial \Omega \:|\: x \in [0,1] \wedge z=3  \right\},\\
\Gamma_{D_2} = \left\{ (x,z) \in \partial \Omega \:|\: x = 2 \wedge z \in [0,1]  \right\},\\
\Gamma_{D} = \Gamma_{D_1} \cup \Gamma_{D_2},\\
\Gamma_{N} = \partial \Omega \setminus \Gamma_D
$$

In [1]:
import shutil
import os

import numpy as np

import porepy as pp
import pygeon as pg

import time

/opt/conda/lib/python3.11/site-packages/porepy/numerics/nonlinear/nonlinear_solvers.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange  # type: ignore


In [2]:
from richards.model_params import Model_Data
from richards.matrix_computer import Matrix_Computer

from richards.solver import Solver
from richards.solver_params import Solver_Data, Solver_Enum, Norm_Error

In [3]:
# Set the maximum number of iterations of the non-linear solver
K = 500

# L-scheme parameter
L = 3.501e-2

# Set the mesh refinment
N = 10

# Set the number of steps (excluding the initial condition)
num_steps = 9

# Simulation time length
T = num_steps/48

# Time switch conditions (for the boundary condition)
dt_D = 1/16

# Fluid density
rho = 1000

# Relative and absolute tolerances for the non-linear solver
abs_tol = 1e-5
rel_tol = 1e-5

# Domain tolerance
domain_tolerance = 1 / (10 * N)

# Output directory
output_directory = 'primal_multi_stage'

In [4]:
# Van Genuchten model parameters ( relative permeability model )
model_data = Model_Data(theta_r=0.131, theta_s=0.396, alpha=0.423, n=2.06, K_s=4.96e-2, T=T, num_steps=num_steps)

In [5]:
# Prepare the domain and its mesh
subdomain = pp.StructuredTriangleGrid([2*N, 3*N], [2,3])
pg.convert_from_pp(subdomain)

# Convert it to a mixed-dimensional grid
mdg = pg.as_mdg(subdomain)

In [6]:
key = "flow"

# Collection of boundary conditions
bc_value = []
bc_essential = []

# Initial pressure
initial_pressure = []

In [7]:
# Initial pressure function
def initial_pressure_func(x): 
    return 1

In [8]:
subdomain, data = mdg.subdomains(return_data=True)[0]

# Gamma_D1 and Gamma_D2 boundary faces
gamma_d1 = np.logical_and(subdomain.nodes[0, :] > 0-domain_tolerance, np.logical_and(subdomain.nodes[0, :] < 1+domain_tolerance, subdomain.nodes[1, :] > 3-domain_tolerance))
gamma_d2 = np.logical_and(subdomain.nodes[0, :] > 2-domain_tolerance, np.logical_and(subdomain.nodes[1, :] > 0-domain_tolerance, subdomain.nodes[1, :] < 1+domain_tolerance))

gamma_d  = np.logical_or(gamma_d1, gamma_d2)
bc_essential = gamma_d

# Add a lambda function that generates for each time instant the (discretized) natural boundary conditions for the problem
bc_value = lambda t: np.array(gamma_d2, dtype=float) + np.array(gamma_d1, dtype=float) * min(3.2, 1 + 2.2 * t / dt_D)

In [9]:
if os.path.exists(output_directory):
    shutil.rmtree(output_directory)

In [10]:
cp = Matrix_Computer(mdg)

solver_data = Solver_Data(mdg=mdg, 
                          initial_solution=cp.P1.interpolate(subdomain, initial_pressure_func), 
                          scheme=Solver_Enum.NEWTON, 
                          bc_essential=lambda t: bc_essential, bc_essential_value=bc_value,
                          eps_psi_rel=rel_tol, eps_psi_abs=abs_tol,
                          max_iterations_per_step=K, L_Scheme_value=L,
                          output_directory=output_directory, report_name='primal',
                          primal=True, integration_order=3, 
                          prepare_plots=False, 
                          shape_x=2*N+1, shape_y=3*N+1, 
                          norm_error=Norm_Error.EUCLIDIAN)

In [11]:
solver = Solver(model_data=model_data, solver_data=solver_data)

In [12]:
start = time.time()
solver.multistage_solver(schemes=[Solver_Enum.LSCHEME], iterations=[15], abs_tolerances=[0], rel_tolerances=[1e-1])
end = time.time()

print('')
print(end - start)

Time 0.02083
LSCHEME


Iteration #001, relative norm of the error:    0.09635, norm of the error:    2.45843

NEWTON
Iteration #0001, relative norm of the error:    0.007690073, norm of the error:    0.199968560
Iteration #0002, relative norm of the error:    0.000207688, norm of the error:    0.005407158
Iteration #0003, relative norm of the error:    0.000001582, norm of the error:    0.000041180

Time 0.04167
LSCHEME
Iteration #001, relative norm of the error:    0.10642, norm of the error:    2.77064
Iteration #002, relative norm of the error:    0.02314, norm of the error:    0.62416

NEWTON
Iteration #0001, relative norm of the error:    0.002247966, norm of the error:    0.060969259
Iteration #0002, relative norm of the error:    0.000027717, norm of the error:    0.000752117
Iteration #0003, relative norm of the error:    0.000000327, norm of the error:    0.000008879

Time 0.0625
LSCHEME
Iteration #001, relative norm of the error:    0.12624, norm of the error:    3.42542
Iteration #002, relative no